In [1]:
#| hide

%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

import sys
import logging
import ipytest
import json
from pathlib import Path


CODE_FOLDER = Path("code")
CODE_FOLDER.mkdir(parents=True, exist_ok=True)
INFERENCE_CODE_FOLDER = CODE_FOLDER / "inference"
INFERENCE_CODE_FOLDER.mkdir(parents=True, exist_ok=True)

sys.path.extend([f"./{CODE_FOLDER}", f"./{INFERENCE_CODE_FOLDER}"])

DATA_FILEPATH = "penguins.csv"

ipytest.autoconfig(raise_on_error=True)

# By default, The SageMaker SDK logs events related to the default
# configuration using the INFO level. To prevent these from spoiling
# the output of this notebook cells, we can change the logging
# level to ERROR instead.
logging.getLogger("sagemaker.config").setLevel(logging.ERROR)

In [2]:
LOCAL_MODE = False

In [3]:
import os

bucket = os.environ["BUCKET"]
role = os.environ["ROLE"]

S3_LOCATION = f"s3://{bucket}/penguins"

In [4]:
architecture = !(uname -m)
IS_APPLE_M_CHIP = architecture[0] == "arm64"

In [5]:
import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession, LocalPipelineSession

pipeline_session = PipelineSession(default_bucket=bucket) if not LOCAL_MODE else None

if LOCAL_MODE:
    config = {
        "session": LocalPipelineSession(default_bucket=bucket),
        "instance_type": "local",
        # We need to use a custom Docker image when we run the pipeline
        # in Local Model on an ARM64 machine.
        "image": "sagemaker-tensorflow-toolkit-local" if IS_APPLE_M_CHIP else None,
    }
else:
    config = {
        "session": pipeline_session,
        "instance_type": "ml.m5.xlarge",
        "image": None,
    }

config["framework_version"] = "2.11"
config["py_version"] = "py39"

In [6]:
import boto3

sagemaker_session = sagemaker.session.Session()
sagemaker_client = boto3.client("sagemaker")
iam_client = boto3.client("iam")
region = boto3.Session().region_name

In [7]:
USE_TUNING_STEP = False and not LOCAL_MODE

In [8]:
# | code: true
# | output: false

# | code: true
# | output: false

from sagemaker.workflow.steps import ProcessingStep
from sagemaker.processing import ProcessingInput, ProcessingOutput
#| code: true
#| output: false
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import FrameworkProcessor
from sagemaker.image_uris import retrieve
from sagemaker.workflow.parameters import ParameterString
from sagemaker.workflow.steps import CacheConfig
from sagemaker.workflow.steps import TrainingStep
from sagemaker.inputs import TrainingInput
from sagemaker.tensorflow import TensorFlow
from sagemaker.workflow.parameters import ParameterInteger

cache_config = CacheConfig(enable_caching=True, expire_after="15d")

# dataset_location = ParameterString(
#     name="dataset_location",
#     default_value=f"{S3_LOCATION}/data",
# )

dataset_location = ParameterString(
    name="dataset_location",
    default_value=f"{S3_LOCATION}/data",
)

new_prep_processor = False

processor = None

if new_prep_processor:
    # Specify the directory containing your `requirements.txt`
    processor = FrameworkProcessor(
        framework_version="1.2-1",
        estimator_cls=SKLearnProcessor,
        base_job_name="preprocess-data",
        image_uri=retrieve(framework='sklearn', version='1.2-1', region=config["session"].boto_region_name),
        command=["python3", "-m", "preprocessor"],
        instance_type=config["instance_type"],
        instance_count=2,
        role=role,
        sagemaker_session=config["session"],
    )

else:
    processor = SKLearnProcessor(
        base_job_name="preprocess-data",
        framework_version="1.2-1",
        # By default, a new account doesn't have access to `ml.m5.xlarge` instances.
        # If you haven't requested a quota increase yet, you can use an
        # `ml.t3.medium` instance type instead. This will work out of the box, but
        # the Processing Job will take significantly longer than it should have.
        # To get access to `ml.m5.xlarge` instances, you can request a quota
        # increase under the Service Quotas section in your AWS account.
        instance_type=config["instance_type"],
        instance_count=2,
        role=role,
        sagemaker_session=config["session"],
    )


preprocessing_step = ProcessingStep(
    name="preprocess-data",
    step_args=processor.run(
        code=f"{CODE_FOLDER}/preprocessor.py",
        inputs=[
            ProcessingInput(
                source=dataset_location, destination="/opt/ml/processing/input", s3_data_distribution_type="ShardedByS3Key"
            )
        ],
        outputs=[
            ProcessingOutput(
                output_name="train",
                source="/opt/ml/processing/train",
                destination=f"{S3_LOCATION}/preprocessing/train",
            ),
            ProcessingOutput(
                output_name="validation",
                source="/opt/ml/processing/validation",
                destination=f"{S3_LOCATION}/preprocessing/validation",
            ),
            ProcessingOutput(
                output_name="test",
                source="/opt/ml/processing/test",
                destination=f"{S3_LOCATION}/preprocessing/test",
            ),
            ProcessingOutput(
                output_name="model",
                source="/opt/ml/processing/model",
                destination=f"{S3_LOCATION}/preprocessing/model",
            ),
            ProcessingOutput(
                output_name="train-baseline",
                source="/opt/ml/processing/train-baseline",
                destination=f"{S3_LOCATION}/preprocessing/train-baseline",
            ),
            ProcessingOutput(
                output_name="test-baseline",
                source="/opt/ml/processing/test-baseline",
                destination=f"{S3_LOCATION}/preprocessing/test-baseline",
            ),
        ],
    ),
    cache_config=cache_config,
)


epochs = ParameterInteger(name="epochs", default_value=50)

estimator = TensorFlow(
    base_job_name="training",
    entry_point=f"{CODE_FOLDER}/train.py",
    # SageMaker will pass these hyperparameters as arguments
    # to the entry point of the training script.
    hyperparameters={
        "epochs": epochs,  # Referencing the pipeline parameter
        "batch_size": 32,
        "learning_rate": 0.01,
    },
    # SageMaker will track these metrics as part of the experiment
    # associated to this pipeline. The metric definitions tells
    # SageMaker how to parse the values from the Training Job logs.
    metric_definitions=[
        {"Name": "loss", "Regex": "loss: ([0-9\\.]+)"},
        {"Name": "accuracy", "Regex": "accuracy: ([0-9\\.]+)"},
        {"Name": "val_loss", "Regex": "val_loss: ([0-9\\.]+)"},
        {"Name": "val_accuracy", "Regex": "val_accuracy: ([0-9\\.]+)"},
    ],
    image_uri=config["image"],
    framework_version=config["framework_version"],
    py_version=config["py_version"],
    instance_type=config["instance_type"],
    instance_count=1,
    disable_profiler=True,
    sagemaker_session=config["session"],
    role=role,
)

train_model_step = TrainingStep(
    name="train-model",
    estimator=estimator,
    inputs={
        "train": TrainingInput(
            s3_data=preprocessing_step.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=preprocessing_step.properties.ProcessingOutputConfig.Outputs["validation"].S3Output.S3Uri,
            content_type="text/csv"
        )
    },
    # cache_config is optional, shown here as part of a complete example
    cache_config=cache_config  # Assuming cache_config is defined elsewhere
)

/Users/jerryobex/proj/mlschool/.venv/lib/python3.9/site-packages/sagemaker/workflow/pipeline_context.py:297: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [9]:
%%writefile {CODE_FOLDER}/evaluation.py
#| label: evaluation-script
#| echo: true
#| output: false
#| filename: evaluation.py
#| code-line-numbers: true

import json
import tarfile
import numpy as np
import pandas as pd

from pathlib import Path
from tensorflow import keras
from sklearn.metrics import accuracy_score, precision_score, recall_score

def evaluate(model_path, test_path, output_path):
    X_test = pd.read_csv(Path(test_path) / "test.csv")
    y_test = X_test.iloc[:, -1]  # Assuming the last column is the target
    island_columns = X_test.columns[-4:-1]  # Last 3 columns are one-hot encoded islands, excluding the target
    X_test.drop(X_test.columns[-1], axis=1, inplace=True)  # Drop target column only

    with tarfile.open(Path(model_path) / "model.tar.gz") as tar:
        tar.extractall(path=Path(model_path))
        
    model = keras.models.load_model(Path(model_path) / "001")
    
    predictions = np.argmax(model.predict(X_test), axis=-1)
    overall_accuracy = accuracy_score(y_test, predictions)
    print(f"Overall Test Accuracy: {overall_accuracy}")

    precision = precision_score(y_test, predictions, average=None, zero_division=0)
    recall = recall_score(y_test, predictions, average=None, zero_division=0)
    
    # Print overall precision and recall
    print("Overall Precision by Class:", precision)
    print("Overall Recall by Class:", recall)

    island_accuracies = {}
    for col in island_columns:
        island_mask = X_test[col] == 1
        y_island = y_test[island_mask]
        if len(y_island) > 0:
            island_predictions = np.argmax(model.predict(X_test[island_mask]), axis=-1)
            island_accuracy = accuracy_score(y_island, island_predictions)
            island_accuracies[col] = island_accuracy
            print(f"Accuracy for island {col}: {island_accuracy}")
        else:
            island_accuracies[col] = None
            print(f"No samples for {col}.")

    evaluation_report = {
        "metrics": {
            "accuracy": {"value": overall_accuracy},
            "island_accuracies": island_accuracies,
            "precision": {f"class_{i}": val for i, val in enumerate(precision)},
            "recall": {f"class_{i}": val for i, val in enumerate(recall)},
        },
    }
    
    Path(output_path).mkdir(parents=True, exist_ok=True)
    with open(Path(output_path) / "evaluation.json", "w") as f:
        f.write(json.dumps(evaluation_report))
        
if __name__ == "__main__":
    evaluate(
        model_path="/opt/ml/processing/model/", 
        test_path="/opt/ml/processing/test/",
        output_path="/opt/ml/processing/evaluation/"
    )


Overwriting code/evaluation.py


In [10]:
%%ipytest -s
#| code-fold: true
#| output: false

import os
import shutil
import tarfile
import pytest
import tempfile
import joblib

from preprocessor import preprocess
from train import train
from evaluation import evaluate


@pytest.fixture(scope="function", autouse=False)
def directory():
    directory = tempfile.mkdtemp()
    input_directory = Path(directory) / "input"
    input_directory.mkdir(parents=True, exist_ok=True)
    shutil.copy2(DATA_FILEPATH, input_directory / "data.csv")
    
    directory = Path(directory)
    
    preprocess(base_directory=directory)
    
    train(
        model_directory=directory / "model",
        train_path=directory / "train", 
        validation_path=directory / "validation",
        epochs=1
    )
    
    # After training a model, we need to prepare a package just like
    # SageMaker would. This package is what the evaluation script is
    # expecting as an input.
    with tarfile.open(directory / "model.tar.gz", "w:gz") as tar:
        tar.add(directory / "model" / "001", arcname="001")
        
    evaluate(
        model_path=directory, 
        test_path=directory / "test",
        output_path=directory / "evaluation",
    )

    yield directory / "evaluation"
    
    shutil.rmtree(directory)


def test_evaluate_generates_evaluation_report(directory):
    output = os.listdir(directory)
    assert "evaluation.json" in output


def test_evaluation_report_contains_accuracy(directory):
    with open(directory / "evaluation.json", 'r') as file:
        report = json.load(file)
        
    assert "metrics" in report
    assert "accuracy" in report["metrics"]
    assert "island_accuracies" in report["metrics"]
    assert "precision" in report["metrics"]
    assert "recall" in report["metrics"]
    

8/8 - 0s - loss: 1.0428 - accuracy: 0.3054 - val_loss: 1.0125 - val_accuracy: 0.3529 - 162ms/epoch - 20ms/step
2/2 [==============================] - 0s 978us/step
Validation accuracy: 0.35294117647058826
INFO:tensorflow:Assets written to: /var/folders/5k/bjy1b7pd2wxctw1dtvx0l2fc0000gn/T/tmpk2u07kyi/model/001/assets
2/2 [==============================] - 0s 1ms/step
Overall Test Accuracy: 0.21568627450980393
Overall Precision by Class: [0.20689655 0.23809524 0.        ]
Overall Recall by Class: [0.26086957 0.5        0.        ]
1/1 [==============================] - 0s 7ms/step
Accuracy for island 0.0: 0.04
1/1 [==============================] - 0s 7ms/step
Accuracy for island 1.0: 0.2777777777777778
1/1 [==============================] - 0s 7ms/step
Accuracy for island 0.0.1: 0.625
.8/8 - 0s - loss: 1.1497 - accuracy: 0.1213 - val_loss: 1.1250 - val_accuracy: 0.1176 - 152ms/epoch - 19ms/step
2/2 [==============================] - 0s 1ms/step
Validation accuracy: 0.11764705882352941
I

### Step 2 - Setting up the Evaluation Step

To run the evaluation script, we will use a [Processing Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing) configured with [TensorFlowProcessor](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job-frameworks-tensorflow.html) because the script needs access to TensorFlow.


One of the inputs to the Evaluation Step will be the model assets. We can use the `USE_TUNING_STEP` flag to determine whether we created the model using a Training Step or a Tuning Step. In case we are using the Tuning Step, we can use the [TuningStep.get_top_model_s3_uri()](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TuningStep.get_top_model_s3_uri) function to get the model assets from the top performing training job of the Hyperparameter Tuning Job.


In [12]:
from sagemaker.workflow.steps import TuningStep
from sagemaker.tuner import HyperparameterTuner
from sagemaker.parameter import IntegerParameter
from sagemaker.tensorflow import TensorFlowProcessor

evaluation_processor = TensorFlowProcessor(
    base_job_name="evaluation-processor",
    image_uri=config["image"],
    framework_version=config["framework_version"],
    py_version=config["py_version"],
    instance_type=config["instance_type"],
    instance_count=1,
    role=role,
    sagemaker_session=config["session"],
)

# target_metric_name = "val_accuracy"
# objective_type="Maximize",
target_metric_name = "loss"
objective_type="Minimize",

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name=target_metric_name,
    objective_type=objective_type,
    hyperparameter_ranges={
        "epochs": IntegerParameter(10, 50),
    },
    metric_definitions=[{"Name": target_metric_name, "Regex": f"{target_metric_name}: ([0-9\\.]+)"}],
    max_jobs=3,
    max_parallel_jobs=3,
)

tune_model_step = TuningStep(
    name="tune-model",
    step_args=tuner.fit(
        inputs={
            "train": TrainingInput(
                s3_data=preprocessing_step.properties.ProcessingOutputConfig.Outputs[
                    "train"
                ].S3Output.S3Uri,
                content_type="text/csv",
            ),
            "validation": TrainingInput(
                s3_data=preprocessing_step.properties.ProcessingOutputConfig.Outputs[
                    "validation"
                ].S3Output.S3Uri,
                content_type="text/csv",
            ),
        },
    ),
    cache_config=cache_config,
)
model_assets = train_model_step.properties.ModelArtifacts.S3ModelArtifacts

if USE_TUNING_STEP:
    model_assets = tune_model_step.get_top_model_s3_uri(
        top_k=0, s3_bucket=config["session"].default_bucket()
    )

/Users/jerryobex/proj/mlschool/.venv/lib/python3.9/site-packages/sagemaker/workflow/pipeline_context.py:297: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


SageMaker supports mapping outputs to property files. This is useful when accessing a specific property from the pipeline. In our case, we want to access the accuracy of the model in the Condition Step, so we'll map the evaluation report to a property file. Check [How to Build and Manage Property Files](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-propertyfile.html) for more information.


We are now ready to define the [ProcessingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep) that will run the evaluation script:

In [14]:
# | code: true
# | output: false

from sagemaker.workflow.properties import PropertyFile

evaluation_report = PropertyFile(
    name="evaluation-report", output_name="evaluation", path="evaluation.json"
)

evaluate_model_step = ProcessingStep(
    name="evaluate-model",
    step_args=evaluation_processor.run(
        inputs=[
            # The first input is the test split that we generated on
            # the first step of the pipeline when we split and
            # transformed the data.
            ProcessingInput(
                source=preprocessing_step.properties.ProcessingOutputConfig.Outputs[
                    "test"
                ].S3Output.S3Uri,
                destination="/opt/ml/processing/test",
            ),
            # The second input is the model that we generated on
            # the Training or Tunning Step.
            ProcessingInput(
                source=model_assets,
                destination="/opt/ml/processing/model",
            ),
        ],
        outputs=[
            # The output is the evaluation report that we generated
            # in the evaluation script.
            ProcessingOutput(
                output_name="evaluation", source="/opt/ml/processing/evaluation"
            ),
        ],
        code=f"{CODE_FOLDER}/evaluation.py",
    ),
    property_files=[evaluation_report],
    cache_config=cache_config,
)

### Step 3 - Registering the Model

Let's now create a new version of the model and register it in the Model Registry. Check [Register a Model Version](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-version.html) for more information about model registration.


Here's a high-level overview of how to register a model in the Model Registry:

<a href="images/registration-step.png" target="_blank"> <img src="images/registration-step.png" alt="High-level overview of the registering a model using the SageMaker's Model Step" style="max-width: 750px;" /></a>


First, let's define the name of the group where we'll register the model:


Let's now create the model that we'll register in the Model Registry. The model we trained uses TensorFlow, so we can use the built-in [TensorFlowModel](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#tensorflow-serving-model) class to create an instance of the model:


When we register a model in the Model Registry, we can attach relevant metadata to it. We'll use the evaluation report we generated during the Evaluation Step to populate the [metrics](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_metrics.ModelMetrics) of this model:


We can use a [Model Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-model) to register the model. Check the [ModelStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.model_step.ModelStep) SageMaker's SDK documentation for more information.

In [18]:
# | code: true
# | output: false

from sagemaker.workflow.model_step import ModelStep
from sagemaker.model_metrics import ModelMetrics, MetricsSource
from sagemaker.tensorflow.model import TensorFlowModel
from sagemaker.workflow.functions import Join
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.lambda_helper import Lambda


MODEL_PACKAGE_GROUP = "penguins"

tensorflow_model = TensorFlowModel(
    model_data=model_assets,
    framework_version=config["framework_version"],
    sagemaker_session=config["session"],
    role=role,
)

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=Join(
            on="/",
            values=[
                evaluate_model_step.properties.ProcessingOutputConfig.Outputs["evaluation"].S3Output.S3Uri,
                "evaluation.json",
            ],
        ),
        content_type="application/json",
    )
)

register_model_step = ModelStep(
    name="register-model",
    step_args=tensorflow_model.register(
        model_package_group_name=MODEL_PACKAGE_GROUP,
        approval_status="Approved",
        model_metrics=model_metrics,
        content_types=["text/csv"],
        response_types=["application/json"],
        inference_instances=[config["instance_type"]],
        transform_instances=[config["instance_type"]],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="TENSORFLOW",
        framework_version=config["framework_version"],
    ),
)

# TODO Lambda Function's Permissions: Ensure the IAM role assumed by your Lambda function 
#  has sufficient permissions to access the SageMaker Model Registry and any other AWS services it interacts with.

# Assuming 'retrieve_best_model_accuracy' is a Lambda function you've set up to get the latest model accuracy
lambda_step = Lambda(
    function_arn="arn:aws:lambda:region:account-id:function:retrieve_best_model_accuracy",
    sagemaker_session=sagemaker_session,
    input_data={
        'best_model_accuracy_default': 0.5
    } 
)

# Use the output from Lambda as the accuracy threshold
accuracy_threshold = JsonGet(
    step_name=lambda_step.name,
    property_file=lambda_step.output(),
    json_path="best_model_accuracy",
)

latest_model_accuracy = JsonGet(
    step_name=evaluate_model_step.name,
    property_file=evaluation_report,
    json_path="metrics.accuracy.value",
)

condition = ConditionGreaterThanOrEqualTo(
    left=latest_model_accuracy, right=accuracy_threshold,
)

fail_step = FailStep(
    name="fail",
    error_message=Join(
        on=" ",
        values=[
            "Execution failed because the model's accuracy was lower than",
            accuracy_threshold,
        ],
    ),
)

condition_step = ConditionStep(
    name="check-model-accuracy",
    conditions=[condition],
    if_steps=[register_model_step] if not LOCAL_MODE else [],
    else_steps=[fail_step],
)


### Step 4 - Setting up a Condition Step

We only want to register a new model if its accuracy exceeds a predefined threshold. We can use a [Condition Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-condition) together with the evaluation report we generated to accomplish this.

Here's a high-level overview of the Condition Step:

<a href="images/condition-step.png" target="_blank"> <img src="images/condition-step.png" alt="High-level overview of the Condition Step" style="max-width: 750px;" /></a>

Let's define a new [Pipeline Parameter](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-parameters.html) to specify the minimum accuracy that the model should reach for it to be registered.


If the model's accuracy is not greater than or equal our threshold, we will send the pipeline to a [Fail Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-fail) with the appropriate error message. Check the [FailStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.fail_step.FailStep) SageMaker's SDK documentation for more information.


We can use a [ConditionGreaterThanOrEqualTo](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.conditions.ConditionGreaterThanOrEqualTo) condition to compare the model's accuracy with the threshold. Look at the [Conditions](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_model_building_pipeline.html#conditions) section in the documentation for more information about the types of supported conditions.


### Step 5 - Creating the Pipeline

We can now define the SageMaker Pipeline and submit its definition to the SageMaker Pipelines service to create the pipeline if it doesn't exist or update it if it does.


In [23]:
# | code: true
# | output: false
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig

pipeline_definition_config = PipelineDefinitionConfig(use_custom_job_prefix=True)

session3_pipeline = Pipeline(
    name="session3-pipeline",
    parameters=[dataset_location, epochs, accuracy_threshold],
    steps=[
        preprocessing_step,
        tune_model_step if USE_TUNING_STEP else train_model_step,
        evaluate_model_step,
        condition_step,
    ],
    pipeline_definition_config=pipeline_definition_config,
    sagemaker_session=config["session"],
)

session3_pipeline.upsert(role_arn=role)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource
Using provided s3_resource


INFO:sagemaker.processing:Uploaded None to s3://mlschools-data-jerryb/session3-pipeline/code/e065caff8e1610ef2e01027f5cf131ad/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://mlschools-data-jerryb/session3-pipeline/code/2c207c809cb0e0e9a1d77e5247f961f9/runproc.sh
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource
Using provided s3_resource


INFO:sagemaker.processing:Uploaded None to s3://mlschools-data-jerryb/session3-pipeline/code/e065caff8e1610ef2e01027f5cf131ad/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://mlschools-data-jerryb/session3-pipeline/code/2c207c809cb0e0e9a1d77e5247f961f9/runproc.sh


{'PipelineArn': 'arn:aws:sagemaker:us-east-2:992382774880:pipeline/session3-pipeline',
 'ResponseMetadata': {'RequestId': '9a60afa6-53d5-4011-8f8d-3acb1232d2b0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9a60afa6-53d5-4011-8f8d-3acb1232d2b0',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '85',
   'date': 'Tue, 13 Feb 2024 01:55:10 GMT'},
  'RetryAttempts': 0}}

We can now start the pipeline:

#| hide

<div class="alert" style="background-color:#0066cc;"><strong>Note:</strong> 
    The <code>%%script</code> cell magic is a convenient way to prevent the notebook from executing a specific cell. If you want to run the cell, comment out the line containing the <code>%%script</code> cell magic.
</div>


In [169]:
#%%script false --no-raise-error
#| eval: false
#| code: true
#| output: false

session3_pipeline.start()

_PipelineExecution(arn='arn:aws:sagemaker:us-east-2:992382774880:pipeline/session3-pipeline/execution/he85psk96p8g', sagemaker_session=<sagemaker.workflow.pipeline_context.PipelineSession object at 0x2d25782e0>)

### Assignments

-   <span style="padding:4px; line-height:30px; background-color: #f2a68a; color: #000;"><strong>Assignment 3.1</strong></span> The evaluation script computes the accuracy of the model and exports it as part of the evaluation report. Extend the evaluation report by adding the precision and the recall of the model on each one of the classes.

-   <span style="padding:4px; line-height:30px; background-color: #f2a68a; color: #000;"><strong>Assignment 3.2</strong></span> Extend the evaluation script to test the model on each island separately. The evaluation report should contain the accuracy of the model on each island and the overall accuracy.

-   <span style="padding:4px; line-height:30px; background-color: #f2a68a; color: #000;"><strong>Assignment 3.3</strong></span> The Condition Step uses a hard-coded threshold value to determine if the model's accuracy is good enough to proceed. Modify the code so the pipeline uses the accuracy of the latest registered model version as the threshold. We want to register a new model version only if its performance is better than the previous version we registered.

-   <span style="padding:4px; line-height:30px; background-color: #f2a68a; color: #000;"><strong>Assignment 3.4</strong></span> The current pipeline uses either a Training Step or a Tuning Step to build a model. Modify the pipeline to use both steps at the same time. The evaluation script should evaluate the model coming from the Training Step and the best model coming from the Tuning Step and output the accuracy and location in S3 of the best model. You should modify the code to register the model assets specified in the evaluation report.

-   <span style="padding:4px; line-height:30px; background-color: #f2a68a; color: #000;"><strong>Assignment 3.5</strong></span> Pipeline steps can encounter exceptions. In some cases, retrying can resolve these issues. For this assignment, configure the Processing Step so it automatically retries the step a maximum of 5 times if it encounters an `InternalServerError`. Check the [Retry Policy for Pipeline Steps](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-retry-policy.html) documentation for more information.